In [ ]:
import networkx as nx

edgotype_train = nx.read_gpickle("data/y2hEdgotyping/edgotype_train.gpickle")
edgotype_val = nx.read_gpickle("data/y2hEdgotyping/edgotype_val.gpickle")
edgotype_test = nx.read_gpickle("data/y2hEdgotyping/edgotype_test.gpickle")

In [ ]:
def getPPILabel(e):
    y = np.zeros(6)
    for i,med in enumerate(["LW_","LWH1_f_", "LWH10_f_",
               "LWH25_f_", "LWA_f_","LWAH1_f_"]):
        s_wt = e[med+"wt"]
        s_mt = e[med+"mt"]
        if not (np.isnan(s_wt) or np.isnan(s_mt)):
            y[i] = s_wt - s_mt
    return y

In [ ]:
mut2Edge = {}
for e in list(edgotype_train.edges(data=True)) + list(edgotype_val.edges(data=True)) + list(edgotype_test.edges(data=True)):
    s = e[2]["Substitution"]
    if s not in mut2Edge:
        mut2Edge[s] = []
    mut2Edge[s].append((*e,getPPILabel(e[2])))

In [ ]:
vals = []
for mut,edges in mut2Edge.items():
    cln_sig = set([e[2]["clinical_significance_mt"] for e in edges])
    scores = [e[-1] for e in edges]
    vals.append({"mutation":mut, "cln_sig":cln_sig,"scores":scores})

In [ ]:
b = []
p = []
vus = []
for v in vals:
    avgScore = np.stack(v["scores"]).mean(0)
    if "Pathogenic" in v["cln_sig"]:
        p.append(avgScore)
    elif "Benign" in v["cln_sig"]:
        b.append(avgScore)
    else:
        vus.append(avgScore)

In [ ]:
bV = np.stack(b)

In [ ]:
pV = np.stack(p)

In [ ]:
vusV = np.stack(vus)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
bins

In [ ]:
fig,ax = plt.subplots(4,6,figsize=(16,6),sharex=True)
for i in range(bV.shape[1]):
    bVals,bins,_ = ax[0,i].hist(bV[:,i],bins=np.arange(-1,4),density=True)
    vusVals,_,_ = ax[1,i].hist(vusV[:,i],bins=bins,density=True)
    pVals,_,_ = ax[2,i].hist(pV[:,i],bins=bins,density=True)
    ax[3,i].plot(bins[:-1],pVals/bVals)